# Final Pipeline - Mohamed MOHAMED EL BECHIR
La pipeline finale pour l'extraction de features et l'entraînement de modèles (Random Forest, Gradient Boosting et XGBoost) dans le cadre du challenge Kaggle (***Cardiac Pathology Prediction***).

## Structure des données

Les données sont organisées dans un dossier `data/` contenant :

- **`Train/`** : Données d'entraînement avec labels, sous forme de sous-dossiers par patient (`001/`, `002/`, etc.), chacun contenant :
  - `XXX_ED.nii` et `XXX_ES.nii` : IRM en end-diastole (ED) et end-systole (ES).
  - `XXX_ED_seg.nii` et `XXX_ES_seg.nii` : Segmentations associées.

- **`Test/`** : Données de test, même structure que `Train/`, mais sans labels.

### Reconstruction des données de test

Le pipeline génère un dossier **`Test2/`** dans (`data/`), où les segmentations du ventricule gauche (VG) sont reconstruites selon la même organisation.

### Métadonnées

- `metadataTrain.csv` : Contient les IDs des patients et leurs labels.
- `metadataTest.csv` : Contient uniquement les IDs des patients.

### Catégories

Les catégories (labels) représentent les diagnostics cardiaques suivants :
- `0` : Healthy controls (Contrôles sains)
- `1` : Myocardial infarction (Infarctus du myocarde)
- `2` : Dilated cardiomyopathy (Cardiomyopathie dilatée)
- `3` : Hypertrophic cardiomyopathy (Cardiomyopathie hypertrophique)
- `4` : Abnormal right ventricle (Ventricule droit anormal)


In [1]:
# ==== INFO VERSIONS ====
import sys
import numpy as np
import pandas as pd
import nibabel as nib
import cv2
import os
from scipy.stats import kurtosis, skew
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from xgboost import XGBClassifier
from scipy import ndimage

print(f"Python : {sys.version.split()[0]}")
print("numpy version:", np.__version__)
print("pandas version:", pd.__version__)
print("nibabel version:", nib.__version__)
print("cv2 version:", cv2.__version__)
print("scipy version:", cv2.__version__)
print("sklearn version:", cv2.__version__)
print("xgboost version:", cv2.__version__)
print("pathlib version:", cv2.__version__)


# -----------------------------------------------------------------------------




Python : 3.11.12
numpy version: 1.26.4
pandas version: 2.2.3
nibabel version: 5.3.2
cv2 version: 4.11.0
scipy version: 4.11.0
sklearn version: 4.11.0
xgboost version: 4.11.0
pathlib version: 4.11.0


In [ ]:
from tqdm import tqdm

def segment_lv_from_myo(seg):
    seg_lv = seg.copy()
    # Remplir les trous slice par slice
    for z in range(seg.shape[2]):
        slice_ = seg[:,:,z]
        myo = (slice_ == 2)
        filled = ndimage.binary_fill_holes(myo)
        lv = filled & (~myo)
        seg_lv[:,:,z][lv] = 3
    return seg_lv

# Dossiers d'entrée et sortie
train_dir = "data/Test"
test_dir = "data/Test2"
os.makedirs(test_dir, exist_ok=True)

# Liste des patients (dossiers)
subject_ids = [d for d in sorted(os.listdir(train_dir)) if d.isdigit()]  # ["101", "102", ...]

for subject_id in tqdm(subject_ids):
    subject_train_path = os.path.join(train_dir, subject_id)
    if not os.path.isdir(subject_train_path):
        continue  # on saute les fichiers

    # Dossier de sortie
    subject_test_path = os.path.join(test_dir, subject_id)
    os.makedirs(subject_test_path, exist_ok=True)

    for phase in ["ED", "ES"]:
        # Fichiers image et segmentation
        img_path = os.path.join(subject_train_path, f"{subject_id}_{phase}.nii")
        seg_path = os.path.join(subject_train_path, f"{subject_id}_{phase}_seg.nii")

        if not os.path.exists(img_path) or not os.path.exists(seg_path):
            print(f"Fichiers manquants pour {subject_id} {phase}")
            continue

        # Charger image (pas modifiée)
        img_nii = nib.load(img_path)
        img_data = img_nii.get_fdata()

        # Charger segmentation et reconstruire le label 3
        seg_nii = nib.load(seg_path)
        seg_data = seg_nii.get_fdata().astype(np.uint8)  
        seg_with_lv = segment_lv_from_myo(seg_data)
        seg_with_lv = seg_with_lv.astype(np.uint8)      

        # Sauvegarde image et segmentation reconstruite
        nib.save(nib.Nifti1Image(img_data, img_nii.affine), os.path.join(subject_test_path, f"{subject_id}_{phase}.nii"))
        nib.save(nib.Nifti1Image(seg_with_lv, seg_nii.affine), os.path.join(subject_test_path, f"{subject_id}_{phase}_seg.nii"))


100%|██████████| 50/50 [00:02<00:00, 18.29it/s]


## Fonctions de Feature Engineering

In [3]:
def compute_binary_volume(mask: np.ndarray, spacing: np.ndarray) -> float:
    """
    Calcule le volume binaire d'un masque 3D (nombre de voxels * volume de chaque voxel).
    """
    return float(mask.sum() * np.prod(spacing))

def compute_wall_thickness(mask3d: np.ndarray, spacing: np.ndarray):
    """
    Calcule l'épaisseur moyenne, écart-type, max et min du 'myocarde' dans chaque slice 2D,
    en utilisant la distance transform de OpenCV.
    """
    scale = np.sqrt(spacing[0] + spacing[1])
    thicknesses = [
        cv2.distanceTransform(mask3d[:, :, z].astype(np.uint8), cv2.DIST_L2, 5).ptp() * scale
        for z in range(mask3d.shape[2])
    ]
    arr = np.asarray(thicknesses)
    return arr.mean(), arr.std(), arr.max(), arr.min()

def compute_wall_circularity(mask3d: np.ndarray, spacing: np.ndarray) -> float:
    """
    Calcule la circularité (4*pi*Surface / Périmètre^2) moyenne sur toutes les slices 2D.
    """
    circ = []
    scale = np.sqrt(spacing[0] + spacing[1])
    for z in range(mask3d.shape[2]):
        contours, _ = cv2.findContours(mask3d[:, :, z].astype(np.uint8),
                                       cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            area = cv2.contourArea(contours[0])
            perimeter = cv2.arcLength(contours[0], True) * scale
            if perimeter:
                circ.append(4 * np.pi * area / perimeter**2)
    return float(np.mean(circ)) if circ else 0.0

def compute_wall_perimeter(mask3d: np.ndarray, spacing: np.ndarray):
    """
    Retourne la moyenne, le max et le min des périmètres du myocarde sur toutes les slices 2D.
    """
    perimeters = []
    scale = np.sqrt(spacing[0] + spacing[1])
    for z in range(mask3d.shape[2]):
        contours, _ = cv2.findContours(mask3d[:, :, z].astype(np.uint8),
                                       cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        perimeters.extend(cv2.arcLength(c, True) * scale for c in contours)
    arr = np.asarray(perimeters)
    if len(arr) == 0:
        return 0.0, 0.0, 0.0
    return arr.mean(), arr.max(), arr.min()

def extract_patient_features(pid: int, root_dir: Path, *, label: int | None = None) -> dict:
    """
    Extrait tous les features (anatomiques, géométriques, fonctionnels, etc.) pour un patient donné.
    """
    sid = f"{pid:03d}"
    seg_ed = nib.load(root_dir / sid / f"{sid}_ED_seg.nii")
    seg_es = nib.load(root_dir / sid / f"{sid}_ES_seg.nii")

    d_ed, d_es = seg_ed.get_fdata().astype(int), seg_es.get_fdata().astype(int)
    v_ed, v_es = np.array(seg_ed.header.get_zooms()), np.array(seg_es.header.get_zooms())

    record = {"Id": pid}
    if label is not None:
        record["Category"] = label

    # --- Anatomy features ---
    anatomy_features = {
        "vol_rv_ED": compute_binary_volume(d_ed == 1, v_ed),
        "vol_myo_ED": compute_binary_volume(d_ed == 2, v_ed),
        "vol_lv_ED": compute_binary_volume(d_ed == 3, v_ed),
        "vol_rv_ES": compute_binary_volume(d_es == 1, v_es),
        "vol_myo_ES": compute_binary_volume(d_es == 2, v_es),
        "vol_lv_ES": compute_binary_volume(d_es == 3, v_es),
    }
    # Ratios
    anatomy_features.update({
        "ratio_rv_lv_ED": anatomy_features["vol_rv_ED"] / anatomy_features["vol_lv_ED"]
                           if anatomy_features["vol_lv_ED"] else 0.0,
        "ratio_myo_lv_ED": anatomy_features["vol_myo_ED"] / anatomy_features["vol_lv_ED"]
                            if anatomy_features["vol_lv_ED"] else 0.0,
        "ratio_rv_lv_ES": anatomy_features["vol_rv_ES"] / anatomy_features["vol_lv_ES"]
                           if anatomy_features["vol_lv_ES"] else 0.0,
        "ratio_myo_lv_ES": anatomy_features["vol_myo_ES"] / anatomy_features["vol_lv_ES"]
                            if anatomy_features["vol_lv_ES"] else 0.0,
    })

    # --- Geometry features ---
    th_mean_ed, th_std_ed, th_max_ed, th_min_ed = compute_wall_thickness(d_ed == 2, v_ed)
    th_mean_es, th_std_es, th_max_es, th_min_es = compute_wall_thickness(d_es == 2, v_es)
    peri_mean_ed, peri_max_ed, peri_min_ed = compute_wall_perimeter(d_ed == 2, v_ed)
    peri_mean_es, peri_max_es, peri_min_es = compute_wall_perimeter(d_es == 2, v_es)

    geometry_features = {
        "th_mean_ED": th_mean_ed,
        "th_std_ED": th_std_ed,
        "th_max_ED": th_max_ed,
        "th_min_ED": th_min_ed,
        "circ_ED": compute_wall_circularity(d_ed == 2, v_ed),
        "peri_mean_ED": peri_mean_ed,
        "peri_max_ED": peri_max_ed,
        "peri_min_ED": peri_min_ed,
        "th_mean_ES": th_mean_es,
        "th_std_ES": th_std_es,
        "th_max_ES": th_max_es,
        "th_min_ES": th_min_es,
        "circ_ES": compute_wall_circularity(d_es == 2, v_es),
        "peri_mean_ES": peri_mean_es,
        "peri_max_ES": peri_max_es,
        "peri_min_ES": peri_min_es,
    }

    # --- Functional features ---
    function_features = {
        "EF_lv": (anatomy_features["vol_lv_ED"] - anatomy_features["vol_lv_ES"])
                  / anatomy_features["vol_lv_ED"] if anatomy_features["vol_lv_ED"] else 0.0,
        "EF_rv": (anatomy_features["vol_rv_ED"] - anatomy_features["vol_rv_ES"])
                  / anatomy_features["vol_rv_ED"] if anatomy_features["vol_rv_ED"] else 0.0,
    }

    # --- Diff features ---
    diff_map = ((d_ed == 2).astype(int) - (d_es == 2).astype(int)).ravel() * np.prod(v_ed)
    diff_features = {
        "diff_med": float(np.median(diff_map)),
        "diff_std": float(np.std(diff_map)),
        "diff_kurt": float(kurtosis(diff_map)),
        "diff_skew": float(skew(diff_map)),
    }

    # Assemblage final
    record.update(anatomy_features)
    record.update(geometry_features)
    record.update(function_features)
    record.update(diff_features)
    return record

def load_features(meta_csv: str, root: str, *, is_train: bool) -> pd.DataFrame:
    """
    Charge toutes les features pour un ensemble de patients (train ou test)
    en itérant sur un fichier CSV de métadonnées.
    """
    meta = pd.read_csv(meta_csv)
    root_dir = Path(root)
    rows = [
        extract_patient_features(int(row.Id), root_dir, label=int(row.Category) if is_train else None)
        for _, row in meta.iterrows()
    ]
    return pd.DataFrame(rows)

## Fonctions pour entraînement et combinaison de modèles

In [4]:
def train_random_forest(X: pd.DataFrame, y: pd.Series) -> RandomForestClassifier:
    """
    Entraîne un RandomForestClassifier avec des hyperparamètres fixés.
    """
    rf = RandomForestClassifier(
        bootstrap=True,
        criterion='gini',
        max_depth=40,
        max_features='sqrt',
        min_samples_leaf=2,
        min_samples_split=8,
        n_estimators=651,
        random_state=0,
        n_jobs=-1
    )
    rf.fit(X, y)
    return rf

def train_gradient_boosting(X: pd.DataFrame, y: pd.Series) -> GradientBoostingClassifier:
    """
    Entraîne un GradientBoostingClassifier basique.
    """
    gb = GradientBoostingClassifier(
        n_estimators=100,
        learning_rate=1,
        max_depth=3,
        random_state=34
    )
    gb.fit(X, y)
    return gb

def train_xgb_classifier(X: pd.DataFrame, y_bin: pd.Series) -> XGBClassifier:
    """
    Entraîne un XGBClassifier sur des labels binaires (1 et 2).
    """
    xgb = XGBClassifier(
        objective="binary:logistic",
        eval_metric="logloss",
        n_estimators=100,
        learning_rate=1,
        max_depth=3,
        random_state=34,
        n_jobs=-1,
    )
    xgb.fit(X, y_bin)
    return xgb

def dynamic_override(pred_gb, proba_gb, gb_classes, pred_xgb, proba_xgb_bin, pred_xgb_bin):
    """
    Remplace la prédiction du GB par celle du XGB si la confiance de XGB est plus élevée.
    """
    idx_gb = {cls: i for i, cls in enumerate(gb_classes)}
    final = pred_gb.copy()
    for i, (pg, px) in enumerate(zip(pred_gb, pred_xgb)):
        if proba_xgb_bin[i, pred_xgb_bin[i]] > proba_gb[i, idx_gb[pg]]:
            final[i] = px
    return final

## Programme Principal

In [5]:
# Liste de features sélectionnées pour la deuxième étape de classification 
# (tels que j'ai definis dans le notebook Feature_Model_Testing).
FEAT2 = [
    'vol_rv_ED', 'vol_myo_ED', 'vol_lv_ED',
    'ratio_rv_lv_ED', 'ratio_myo_lv_ED',
    'vol_rv_ES', 'vol_myo_ES', 'vol_lv_ES',
    'th_mean_ES', 'ratio_rv_lv_ES',
    'ratio_myo_lv_ES', 'EF_lv', 'EF_rv'
]
def main():
    """
    Exécute l'ensemble du pipeline :
    1) Extraction des features
    2) Entraînement d'un premier modèle (RandomForest)
    3) Sélection des patients (cat. 1 ou 2) à affiner
    4) Entraînement de modèles de niveau 2 (GB, XGB)
    5) Fusion des prédictions
    6) Sauvegarde du fichier submission.csv
    """
    # 1. Feature extraction
    df_train = load_features("metadataTrain.csv", "data/Train", is_train=True)
    df_test  = load_features("metadataTest.csv",  "data/Test2",  is_train=False)

    # Séparation features / labels pour l'entraînement
    X, y = df_train.drop(columns=["Id", "Category"]), df_train["Category"].astype(int)
    X_ts = df_test.drop(columns=["Id"])

    # 2. Random Forest de premier niveau
    rf = train_random_forest(X, y)
    pred_lvl1 = rf.predict(X_ts)

    # 3. Focus sur les catégories {1, 2}
    mask_tr, mask_ts = y.isin([1, 2]), np.isin(pred_lvl1, [1, 2])
    X2_tr_gb  = X.loc[mask_tr, FEAT2]
    X2_ts_gb  = X_ts.loc[mask_ts, FEAT2]
    X2_tr_xgb = X.loc[mask_tr, FEAT2]
    X2_ts_xgb = X_ts.loc[mask_ts, FEAT2]
    y2_tr     = y[mask_tr]

    # 4. Second niveau : entraînement et prédictions avec GB + XGB
    gb2 = train_gradient_boosting(X2_tr_gb, y2_tr)
    proba_gb = gb2.predict_proba(X2_ts_gb)
    pred_gb  = gb2.predict(X2_ts_gb)

    y2_tr_bin = y2_tr.map({1: 0, 2: 1})
    xgb2 = train_xgb_classifier(X2_tr_xgb, y2_tr_bin)
    proba_xgb_bin = xgb2.predict_proba(X2_ts_xgb)
    pred_xgb_bin  = xgb2.predict(X2_ts_xgb)
    pred_xgb      = np.where(pred_xgb_bin == 0, 1, 2)

    # 5. Fusion finale des prédictions
    pred_final_subset = dynamic_override(pred_gb, proba_gb, gb2.classes_,
                                         pred_xgb, proba_xgb_bin, pred_xgb_bin)
    final_pred = pred_lvl1.copy()
    final_pred[mask_ts] = pred_final_subset

    # 6. Export en CSV
    pd.DataFrame({
        "Id": df_test["Id"].astype(int),
        "Category": final_pred.astype(int)
    }).to_csv("submission.csv", index=False)
    print("submission.csv saved.")

In [6]:
if __name__ == "__main__":
    main()

submission.csv saved.
